In [133]:
import pandas as pd
from math import sqrt
import datetime
import numpy as np
import re

In [275]:
usersDF = pd.read_csv('../../../Downloads/myanimelist/users_cleaned.csv')
animesDF = pd.read_csv('../../../Downloads/myanimelist/animesDF.csv')
scoresDF = pd.read_csv('../../../Downloads/myanimelist/scores.csv')

In [300]:
print(usersDF.shape, animesDF.shape, scoresDF.shape)

(108711, 17) (1711, 33) (473523, 4)


In [298]:
anime_ids = np.sort(animesDF.anime_id.unique())
len(anime_ids)

1711

In [288]:
top_1000_users = list(usersDF.sort_values(by=['user_completed'], ascending=False).head(1000)['username'])

In [289]:
#scoresDF = scoresDF.sample(n=10000, random_state=15)
#scoresDF.to_csv('../../../Downloads/myanimelist/scores_sample.csv', index=False)

In [290]:
scoresDF = scoresDF[scoresDF['username'].isin(top_1000_users)]

In [299]:
scoresDF = scoresDF[scoresDF['anime_id'].isin(anime_ids)]

In [291]:
animesDF = animesDF[(animesDF['duration_min']>=24) \
                    & (animesDF['aired_from_year']>=2000) \
                    & ~(animesDF.related.str.contains('Prequel')) \
                    & ~(animesDF.genre.str.contains('Kids')) \
                    & ~(animesDF.genre.str.contains('Shoujo Ai')) \
                    & ~(animesDF.genre.str.contains('Shounen Ai')) \
                    & ~(animesDF.genre.str.contains('Yaoi')) \
                    & ~(animesDF.genre.str.contains('Yuri'))]
animesDF.to_csv('../../../Downloads/myanimelist/animesDF.csv', index=False)

In [295]:
animesDF.sort_values(by=['anime_id']).head()

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,broadcast,related,producer,licensor,studio,genre,opening_theme,ending_theme,duration_min,aired_from_year
1327,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ天国の扉,Cowboy Bebop: Knockin&#039; on Heaven&#039;s Door,https://myanimelist.cdn-dena.com/images/anime/...,Movie,Original,1,Finished Airing,...,NaN,"{'Parent story': [{'mal_id': 1, 'type': 'anime...","Sunrise, Bandai Visual",Sony Pictures Entertainment,Bones,"Action, Space, Drama, Mystery, Sci-Fi","['""Ask DNA"" by The Seatbelts featuring Raju Ra...","['""Gotta Knock a Little Harder"" by The Seatbel...",114.0,2001.0
1217,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN,WHR,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,26,Finished Airing,...,Tuesdays at Unknown,[],Bandai Visual,"Funimation, Bandai Entertainment",Sunrise,"Action, Magic, Police, Supernatural, Drama, My...","['""Shell"" by Bana']","['#1: ""Half Pain"" by Bana (eps 01-14,16-25)', ...",25.0,2002.0
925,19,Monster,Monster,モンスター,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,74,Finished Airing,...,Wednesdays at 00:40 (JST),"{'Adaptation': [{'mal_id': 1, 'type': 'manga',...","VAP, Shogakukan Productions, Nippon Television...",Viz Media,Madhouse,"Drama, Horror, Mystery, Police, Psychological,...","['""Grain"" by Kuniaki Haishima']","['#1: ""For The Love of Life"" by David Sylvian ...",24.0,2004.0
1171,23,Ring ni Kakero 1,NaN,リングにかけろ１,Put it all in the Ring,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 14231, 'type': 'man...",NaN,NaN,Toei Animation,"Action, Shounen, Sports","['""Asu eno Toushi"" by Marina del ray']","['""TAKE MY SOUL FOREVER"" by Psychic Lover']",25.0,2004.0
338,25,Sunabouzu,Desert Punk,砂ぼうず,Sunabozu,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,24,Finished Airing,...,Unknown,"{'Adaptation': [{'mal_id': 10016, 'type': 'man...","GDH, Pony Canyon, CBC",Funimation,Gonzo,"Action, Adventure, Comedy, Ecchi, Sci-Fi, Shounen","['#1: ""Sand Mission"" by ZETKI (eps 1-11)', '#2...","['#1: ""Sunabouzu Ekaki Uta"" by Hideaki Takator...",24.0,2004.0


In [301]:
scoresDF.head()

,username,anime_id,my_score,my_status
866,MistButterfly,59,6,2
867,MistButterfly,74,7,2
868,MistButterfly,120,8,2
869,MistButterfly,178,5,2
872,MistButterfly,249,7,2


In [306]:
anime_users = scoresDF.pivot(index='anime_id', columns='username', values='my_score')

In [231]:
'''
gets all user scores
'''
def get_user_scores(scoresDF, viewer):
    return scoresDF.loc[(scoresDF['username'] == viewer)]

get_viewer(scoresDF, 'FadeKyun').head()

,username,anime_id,my_score,my_status
3,FadeKyun,20,3,4
37892,FadeKyun,1482,5,4
45938,FadeKyun,14749,9,2
67454,FadeKyun,477,8,2
71844,FadeKyun,1818,5,4


In [285]:
def sim_pearson(scores, anime1_id, anime2_id):
    '''Return the Pearson correlation coefficient for anime1 and anime2.'''
    scores_anime1 = scores[scores.anime_id == anime1_id]
    scores_anime2 = scores[scores.anime_id == anime2_id]
    # get the list of shared viewers
    shared_viewers = 0
    scores1 = np.array([])
    scores2 = np.array([])
    count = 0
    n = scores_anime1.shape[0]
    for index, row in scores_anime1.iterrows():
        viewer = get_user_scores(scores_anime2, row['username'])
        if not viewer.empty:
            shared_viewers += 1
            scores1 = np.append(scores1, row['my_score'])
            scores2 = np.append(scores2, int(viewer['my_score']))
        count += 1
        if count == n:
            print('Viewers cheked: {} out of {} :: Shared: {} :: {}'.format(count, n, shared_viewers, datetime.datetime.now()))
            
    # if no ratings in common, return 0
    if shared_viewers == 0:
        return 0

    # add up all the preferences
    sum1 = np.sum(scores1)
    sum2 = np.sum(scores2)
    
    # sum up the squares
    sum1Sq = np.sum(scores1**2)
    sum2Sq = np.sum(scores2**2)
    
    # sum up the products
    #pSum = sum([score1*score2 for score1, score2 in zip(scores1, scores2)])
    pSum = np.dot(scores1, scores2)
    # calculate Pearson score
    num = pSum - (sum1 * sum2)/shared_viewers
    den = sqrt((sum1Sq - pow(sum1, 2)/shared_viewers) * (sum2Sq - pow(sum2, 2)/shared_viewers))
    if den == 0:
        return 0
    else:
        return num/den

In [286]:
sim_pearson(scoresDF, 31964, 20)

Viewers cheked: 670 out of 670 :: Shared: 553 :: 2020-02-02 18:50:35.488033


0.37532549212144195

In [268]:
'''
builds table of similarity of the shows
a[i][j] - similarity of the shows with ids: anime_ids[i-1] and anime_ids[j-1]
'''
def get_similarity_table(scores, anime_ids):
    similarity_table = np.zeros((len(anime_ids)+1, len(anime_ids)+1))
    similarity_table[1:, 0] = anime_ids
    similarity_table[0, 1:] = anime_ids
    n = len(anime_ids)
    count = 0
    for anime1_id in anime_ids[:1]:
        for anime2_id in anime_ids:
            if anime1_id != anime2_id:
                similarity = sim_pearson(scores, anime1_id, anime2_id)
                x = np.where(similarity_table == anime1_id)[0][1]
                y = np.where(similarity_table == anime2_id)[0][1]
                similarity_table[x][y] = similarity
                similarity_table[y][x] = similarity
        np.save('similarity_table', similarity_table)
        count += 1
        print('Processed {} of {} animes :: {}'.format(count, n, datetime.datetime.now()))
    return similarity_table

In [269]:
print(datetime.datetime.now())
get_similarity_table(scoresDF, anime_ids)

2020-02-02 18:42:26.998525


KeyboardInterrupt: 

In [247]:
test = Out[246]

In [250]:
np.unique(test[1])

array([0., 5.])

In [28]:
'''
loads user's scores from MAL xml file converted to csv
'''
def load_user_list(path):
    userDF = pd.read_csv(path)
    return userDF[['series_animedb_id', 'my_score']][userDF['my_score']!=0]

In [29]:
cmlimmDF = load_user_list('../../../Downloads/animelist_cmlimm.csv')

In [30]:
'''
find user's shows that exist in MAL dataset, as some shows aired after dataset was created
'''
def user_shows_in_db(userDF, animesDF):
    return userDF[userDF[['series_animedb_id']].isin(animesDF.anime_id.unique()).any(1)]

In [31]:
np.array(user_shows_in_db(cmlimmDF, animesDF))

array([[32977,     7],
       [22147,     9],
       [35015,     7],
       [ 2251,    10],
       [ 3901,     8],
       [35965,     7],
       [28805,     7],
       [32664,     6],
       [31964,     8],
       [33486,    10],
       [35262,     5],
       [36456,     9],
       [33929,     8],
       [35459,     8],
       [16592,     6],
       [35849,     8],
       [30346,     7],
       [ 6746,     8],
       [ 9314,     7],
       [18507,     7],
       [22265,     8],
       [26213,     8],
       [19671,     7],
       [35198,     8],
       [33731,     5],
       [ 2890,     7],
       [18229,     7],
       [20583,     9],
       [28891,     9],
       [32935,     9],
       [25303,     7],
       [35806,     7],
       [27989,    10],
       [31988,    10],
       [34204,     7],
       [31665,     7],
       [31096,     7],
       [30415,     8],
       [33241,     6],
       [10357,     9],
       [ 5680,     6],
       [ 9617,     6],
       [ 7791,     6],
       [ 97

In [32]:
'''
finds top matches based on shows similarities
'''
def top_matches(similarity_table, user_show_scores):
    result = np.empty((0,2), int)
    user_shows = user_show_scores[:, 0]
    user_scores = user_show_scores[:, 1]
    to_delete = []
    for show in user_shows:
        y = np.where(similarity_table == show)[0][1]
        to_delete.append(y)
    similarity_table = np.delete(similarity_table, to_delete, 1)
    to_delete = []
    for show in similarity_table[1:, 0]:
        if show not in user_shows:
            x = np.where(similarity_table == show)[0][1]
            to_delete.append(x)
    similarity_table = np.delete(similarity_table, to_delete, 0)
    for column in similarity_table.T[1:]:
        result = np.append(result, [[column[0], np.dot(column[1:], user_scores)/np.sum(column[1:])]], axis=0)
    return result[result[:,1].argsort()][::-1]

In [33]:
def show_names(matches, animesDF):
    for show in matches:
        print(animesDF[animesDF['anime_id']==show[0]]['title'].values[0])